### Establishing BigQuery Connection

Includes account key via the json file stored in the folder which contains the project repo

In [3]:
from google.cloud import bigquery

import pathlib # __file__ isn't recognized in Jupyter, so we need this to get the root path
path = pathlib.Path.cwd()
PROJ_ROOT = path.parent

accountkeyfile = str(PROJ_ROOT.parent / 'credentials\GoogleServiceAccountKeyFile.json')
client = bigquery.Client.from_service_account_json(accountkeyfile)

#### Accounts Per Device

Device determined by device.advertising_id

In [2]:
query = """
SELECT
    device.advertising_id AS device_id,
    COUNT(DISTINCT user_properties.value.string_value) AS Usernames,
    COUNT(DISTINCT user_pseudo_id) AS Pseudo_IDs
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE 
    ((SELECT event_params.value.string_value
        FROM UNNEST(event_params) AS event_params
        WHERE event_params.key = 'firebase_screen_class') = 'HomeActivity')
GROUP BY
    device.advertising_id
ORDER BY 
    COUNT(DISTINCT user_properties.value.string_value) DESC
"""
df = client.query(query).to_dataframe()
df.head()

,device_id,Usernames,Pseudo_IDs
0,733bdd53-745a-4e95-b32e-9595690aaf52,20,29
1,788dc562-cc05-44c0-bc26-7e24ff3c3768,16,36
2,42479d6a-a2e2-4530-be2f-1cc0a77245f0,3,4
3,b3c6b09d-e8b0-413d-a285-bce02fdfb58c,2,2
4,2f84376a-841a-4d6f-83fc-2d339294e02d,2,2


### Total # of Events by Phone model

In [12]:
query = """
SELECT
    COUNT(device.mobile_os_hardware_model) AS events,
    device.mobile_os_hardware_model AS model
FROM
    `heycharlie-ada47.analytics_153084895.events_*`
WHERE
    event_date > '20181109'
GROUP BY
    device.mobile_os_hardware_model
ORDER BY
    COUNT(device.mobile_os_hardware_model) DESC
"""
df = client.query(query).to_dataframe()
df.head()

,events,model
0,103534,SM-N920V
1,40713,Pixel XL
2,40450,SAMSUNG-SM-G891A
3,31763,SM-G930V
4,31462,SM-G950U


### Total # of Events by Username

In [14]:
query = """
SELECT
    COUNT(user_properties.value.string_value) AS events,
    user_properties.value.string_value AS Username
FROM
    `heycharlie-ada47.analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE
    event_date > '20181109'
GROUP BY
    user_properties.value.string_value
ORDER BY
    COUNT(user_properties.value.string_value) DESC
"""
df = client.query(query).to_dataframe()
df.head()

,events,Username
0,51762,Rabbit
1,19876,emily
2,15674,Jake
3,11054,anonysauce
4,8495,mckay68


### # of Notification Events by Username

TODO - Break out notifications by type (positive reinforcement 1x/day, texts, calls)

TODO - Look at responses to notifications

In [19]:
query = """
SELECT
    user_properties.value.string_value as username,
    COUNT(*) AS count
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties,
    UNNEST(event_params) AS event_params
WHERE 
    event_params.key = 'notification_event'
    AND event_date > '20181101'
    AND user_properties.value.string_value != 'null'
GROUP BY
    user_properties.value.string_value
ORDER BY
    COUNT(*) DESC
"""
df = client.query(query).to_dataframe()
df.head()

,username,count
0,mckay68,1296
1,emily,384
2,+vinny,264
3,Jake,198
4,emily3,186


In [21]:
query = """
SELECT
    event_name AS event_name,
    COUNT(event_name) AS count
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties,
    UNNEST(event_params) AS event_params
WHERE 
    event_params.key = 'notification_event'
    AND event_date > '20181101'
    AND user_properties.value.string_value = 'mckay68'
GROUP BY
    event_name
ORDER BY
    COUNT(event_name) DESC
"""
df = client.query(query).to_dataframe()
df.head()

,event_name,count
0,geo_risky_warning_notification,1225
1,friend_scoring_notification,46
2,risky_sent_sms_warning_notification,12
3,risky_sms_warning_notification,6
4,daily_task_notification,2


### Daily Count of Risky Text Events

In [19]:
query = """
SELECT
    event_date AS Date,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_event" THEN 1 ELSE 0 END) AS risky_sent_sms,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_sent_notification,
    SUM(CASE WHEN event_name = "risky_sms_event" THEN 1 ELSE 0 END) AS risky_in_sms,
    SUM(CASE WHEN event_name = "risky_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_in_notification,
    COUNT(event_name) AS events
FROM 
    (SELECT DISTINCT event_date, event_name, event_server_timestamp_offset
        FROM `analytics_153084895.events_*`)
WHERE
    event_name = "risky_sent_sms_warning_event"
    OR event_name = "risky_sent_sms_warning_notification"
    OR event_name = "risky_sms_event"
    OR event_name = "risky_sms_warning_notification"
GROUP BY
    event_date
ORDER BY
    event_date DESC
"""
df = client.query(query).to_dataframe()
df.head()

,Date,risky_sent_sms,risky_sent_notification,risky_in_sms,risky_in_notification,event
0,20190123,1,1,1,1,4
1,20190119,6,4,6,4,20
2,20190118,0,0,1,1,2
3,20190117,4,3,7,7,21
4,20190116,3,3,5,3,14


### Daily Risky Events and Notifications, by User

In [22]:
query = """
SELECT
    event_date AS Date,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_event" THEN 1 ELSE 0 END) AS risky_sent_sms,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_sent_notification,
    SUM(CASE WHEN event_name = "risky_sms_event" THEN 1 ELSE 0 END) AS risky_in_sms,
    SUM(CASE WHEN event_name = "risky_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_in_notification,
    user_properties.value.string_value AS username,
    COUNT(event_name) AS total_risly_sms_events
FROM 
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE
    user_properties.value.string_value <> "null"
    AND (event_name = "risky_sent_sms_warning_event"
        OR event_name = "risky_sent_sms_warning_notification"
        OR event_name = "risky_sms_event"
        OR event_name = "risky_sms_warning_notification")
GROUP BY
    event_date, user_properties.value.string_value
ORDER BY
    event_date DESC
"""
df = client.query(query).to_dataframe()
df.head(10)

,Date,risky_sent_sms,risky_sent_notification,risky_in_sms,risky_in_notification,username,events
0,20190123,3,1,15,3,emily,22
1,20190119,18,4,28,4,vinnov10,54
2,20190118,0,0,1,1,Jake,2
3,20190117,0,0,5,4,vinnov10,9
4,20190117,3,2,22,5,emily,32
5,20190117,1,1,2,0,hbickers,4
6,20190116,8,4,60,4,emily,76
7,20190116,1,1,6,0,liamkl,8
8,20190115,1,0,6,2,hbickers,9
9,20190115,4,3,14,2,emily,23


### Days of App Use

In [ ]:
query = """
SELECT
    event_date AS Date,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_event" THEN 1 ELSE 0 END) AS risky_sent_sms,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_sent_notification,
    SUM(CASE WHEN event_name = "risky_sms_event" THEN 1 ELSE 0 END) AS risky_in_sms,
    SUM(CASE WHEN event_name = "risky_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_in_notification,
    user_properties.value.string_value AS username,
    COUNT(event_name) AS total_risly_sms_events
FROM 
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE
    user_properties.value.string_value <> "null"
    AND (event_name = "risky_sent_sms_warning_event"
        OR event_name = "risky_sent_sms_warning_notification"
        OR event_name = "risky_sms_event"
        OR event_name = "risky_sms_warning_notification")
GROUP BY
    event_date, user_properties.value.string_value
ORDER BY
    event_date DESC
"""
df = client.query(query).to_dataframe()
df.head(10)

### Days of App Use

In [23]:
query = """
SELECT
    MIN(event_date) AS Start_Date,
    MAX(event_date) AS Max_Date,
    ROUND((MAX(CAST(event_timestamp AS INT64)) - MIN(CAST(event_timestamp AS INT64))) / (86400 * 1000000),1) AS Installed_Days,
    user_properties.value.string_value AS Username,
    MIN(user_first_touch_timestamp) = MIN(event_timestamp) AS first_touch_equal_first_event,
    MIN(user_first_touch_timestamp) AS first_touch_timestamp,
    MIN(event_timestamp) AS min_event_timestamp,
    MAX(event_timestamp) AS max_timestamp
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE
    user_first_touch_timestamp > 20181110
GROUP BY
    user_properties.value.string_value
ORDER BY
    (MAX(CAST(event_timestamp AS INT64)) - MIN(CAST(event_timestamp AS INT64))) DESC
"""
df = client.query(query).to_dataframe()
df.head(10)

,Min_Date,Max_Date,Active_Days,Username,first_touch_equal_first_event,first_touch_timestamp,min_event_timestamp,max_timestamp
0,20180711,20190123,196.2,None,False,1524757617404000,1531305681399000,1548257649805857
1,20180806,20181217,133.1,test,False,1524798650061000,1533604351828000,1545100704746055
2,20180917,20190122,127.0,test23,False,1536522983060000,1537238931729000,1548213752564001
3,20180918,20190123,127.0,emily,False,1537276848519000,1537276864561000,1548251351687729
4,20180919,20190123,126.1,liamkl,False,1537361098120000,1537361121551000,1548255184682384
5,20181108,20190122,75.4,chshanahan,False,1541689883839000,1541689955151000,1548207774634011
6,20181110,20190122,73.2,vinnov10,False,1541890948880000,1541891256054000,1548214997130003
7,20180906,20181117,71.9,test5,False,1536287952446000,1536290491856000,1542506112545066
8,20181112,20190123,71.6,Jake,False,1542041638325000,1542041672191000,1548225102724130
9,20181114,20190117,64.3,+vinny,False,1542208044439000,1542208107623000,1547767715989016


### Installs

In [5]:
query = """
SELECT
    MIN(event_date) AS Date,
    MIN(event_timestamp) AS min_event_timestamp,
    MIN(user_first_touch_timestamp) AS Install,
    user_properties.value.string_value AS Username
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
GROUP BY
    user_properties.value.string_value
ORDER BY 
    MIN(event_date) DESC
"""
df = client.query(query).to_dataframe()
df.head(10)

,Date,min_event_timestamp,Install,Username
0,20190123,1548298508164000,1542476516676000,test3
1,20190122,1548207773051000,1548207688773000,Chris
2,20190116,1547640467143000,1547638488374000,rosan
3,20190110,1547164215615000,1542476516676000,google1
4,20190110,1547137784596000,1547137724581000,hbickers
5,20190107,1546885809504000,1546885564003000,yuseff7
6,20190105,1546688785657000,1546688739845000,kaltmann19
7,20190103,1546538000253000,1546537939532000,mattiet
8,20181225,1545790298984000,1545789983494000,bebeqq
9,20181225,1545790025436000,1545789983494000,bebeq


### Event Count For Date Range

TABLE_DATE_RANGE is legacy SQL and seems to only work in the BigQuery web portal

In [16]:
query = """
SELECT
    COUNT(event_name) AS NotificationEvents,
    event_name,
    event_params.value.string_value AS string_value
FROM
    `analytics_153084895.*`,
    UNNEST(event_params) AS event_params
WHERE
    _TABLE_SUFFIX IN ('events_20190122', 'events_20191231')
GROUP BY
    event_name, string_value
ORDER BY
    NotificationEvents DESC
"""

# This is the legacy SQL
# query = """ 
# SELECT
#     COUNT(event_name) AS NotificationEvents,
#     event_name,
#     event_params.value.string_value AS string_value
# FROM
#     TABLE_DATE_RANGE( analytics_153084895.events_,
#         TIMESTAMP('2018-01-01'),
#         TIMESTAMP('2018-12-31'))
# GROUP BY
#     event_name, string_value
# ORDER BY
#     NotificationEvents DESC
# """

df = client.query(query).to_dataframe()
df.head(10)

,NotificationEvents,event_name,string_value
0,449,text_message_received,app
1,449,text_message_received,None
2,448,reviver_scimitar_already_running,app
3,400,text_message_received,sms
4,252,data_sync_friend_event_3_tries,app
5,223,text_message_received,received
6,223,text_message_received,SMSMonitoringService
7,194,text_message_sent,app
8,194,text_message_sent,sms
9,177,text_message_received,BroadcastReceiver


### First Week Feedback

In [18]:
query = """
SELECT
    event_date AS date,
    event_timestamp AS timestamp,
    user_properties.value.string_value as username,
    (SELECT event_params.value.string_value
    FROM UNNEST(event_params) AS event_params
    WHERE event_params.key = 'face') AS face,
    (SELECT event_params.value.string_value
    FROM UNNEST(event_params) AS event_params
    WHERE event_params.key = 'feedback') AS feedback
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE 
    event_name = 'first_week_questionnaire_answer'
    AND user_properties.value.string_value IS NOT NULL
ORDER BY
    event_timestamp DESC
"""
df = client.query(query).to_dataframe()
df

,date,timestamp,username,face,feedback
0,20190115,1547606474382004,+vinny,nuetral,Gabagaba Goo!
1,20190108,1546959287604080,vinnov10,nuetral,Fg\n
2,20190107,1546917550772016,test23,happy,
3,20190107,1546914239980004,vinnov10,happy,Ffv
4,20190107,1546906391776069,vinnov10,happy,Great!
5,20190104,1546639224053006,+vinny,sad,Same day as reinstall\n
6,20190102,1546470188021004,+vinny,nuetral,"Just reinstalled, old login, already got one o..."
7,20190101,1546394111869010,mckay68,happy,None
8,20181205,1544047876047019,+vinny,happy,Test entry
9,20181128,1543445020495005,Upstate518,happy,None


### Onboarding Events

In [22]:
query = """
SELECT
    t1.event_date AS Date,
    t1.event_name AS event_name,
    t1.user_pseudo_id AS user_pseudo_id,
    t2.username as Username,
    t1.event_timestamp AS timestamp,
    t1.event_previous_timestamp AS previous_timestamp,
    t1.user_first_touch_timestamp AS first_touch_timestamp,
    t2.done_timestamp AS done_timestamp,
    (SELECT event_params.value.string_value
        FROM UNNEST(t1.event_params) AS event_params
        WHERE event_params.key = 'firebase_screen_class') AS screen,
    (SELECT event_params.value.string_value
        FROM UNNEST(t1.event_params) AS event_params
        WHERE event_params.key = 'firebase_previous_class') AS prev_screen
FROM
    `analytics_153084895.events_*` as t1,
    UNNEST(t1.user_properties) AS user_properties
JOIN(
    SELECT
        MIN(event_timestamp) AS done_timestamp,
        user_properties.value.string_value AS username
    FROM
        `analytics_153084895.events_*`,
         UNNEST(user_properties) AS user_properties
    WHERE 
        ((SELECT event_params.value.string_value
            FROM UNNEST(event_params) AS event_params
            WHERE event_params.key = 'firebase_screen_class') = 'HomeActivity')
    GROUP BY
        user_properties.value.string_value) AS t2
ON t2.username = user_properties.value.string_value
WHERE
    t1.event_timestamp < t2.done_timestamp
    AND t1.event_name = 'screen_view'
    AND t2.username != 'null'
"""
df = client.query(query).to_dataframe()
df.head(10)

,Date,event_name,user_pseudo_id,Username,timestamp,previous_timestamp,first_touch_timestamp,done_timestamp,screen,prev_screen
0,20180909,screen_view,737f4d6f64fe8b7c9e00640bd7798484,test7,1536521600463002,1536521600430002,1536287952446000,1536521650806029,WelcomeActivity,WelcomeActivity
1,20180909,screen_view,737f4d6f64fe8b7c9e00640bd7798484,test7,1536521605509005,1536521605377005,1536287952446000,1536521650806029,LegalAbstractActivity,LegalAbstractActivity
2,20180909,screen_view,737f4d6f64fe8b7c9e00640bd7798484,test7,1536521607855008,1536521607794008,1536287952446000,1536521650806029,PermissionsActivity,PermissionsActivity
3,20180909,screen_view,737f4d6f64fe8b7c9e00640bd7798484,test7,1536521609414010,1536521609355010,1536287952446000,1536521650806029,NotificationsPermissionActivity,NotificationsPermissionActivity
4,20180909,screen_view,737f4d6f64fe8b7c9e00640bd7798484,test7,1536521610492012,1536521610430012,1536287952446000,1536521650806029,AboutYouActivity,AboutYouActivity
5,20180909,screen_view,737f4d6f64fe8b7c9e00640bd7798484,test7,1536521619324015,1536521619305015,1536287952446000,1536521650806029,PlacesMessageActivity,PlacesMessageActivity
6,20180909,screen_view,737f4d6f64fe8b7c9e00640bd7798484,test7,1536521620852017,1536521620814017,1536287952446000,1536521650806029,PlacesActivity,PlacesActivity
7,20180909,screen_view,737f4d6f64fe8b7c9e00640bd7798484,test7,1536521623226020,1536521623183020,1536287952446000,1536521650806029,RelationshipsMessageActivity,RelationshipsMessageActivity
8,20180909,screen_view,737f4d6f64fe8b7c9e00640bd7798484,test7,1536521624283022,1536521624244022,1536287952446000,1536521650806029,RelationshipsActivity,RelationshipsActivity
9,20180909,screen_view,737f4d6f64fe8b7c9e00640bd7798484,test7,1536521641298025,1536521641271025,1536287952446000,1536521650806029,QuestionsActivity,QuestionsActivity


### Onboarding Screen View Timing

In [23]:
query = """
SELECT
    t1.event_date AS Date,
    t1.event_name AS event_name,
    t1.user_pseudo_id AS user_pseudo_id,
    t2.username as Username,
    t1.event_timestamp AS timestamp,
    t1.event_previous_timestamp AS previous_timestamp,
    t1.user_first_touch_timestamp AS first_touch_timestamp,
    t2.done_timestamp AS done_timestamp,
    (SELECT event_params.value.string_value
        FROM UNNEST(t1.event_params) AS event_params
        WHERE event_params.key = 'firebase_previous_class') AS Screen
FROM
    `analytics_153084895.events_*` as t1,
    UNNEST(t1.user_properties) AS user_properties
JOIN(
    SELECT
        MIN(event_timestamp) AS done_timestamp,
        user_properties.value.string_value AS username
    FROM
        `analytics_153084895.events_*`,
        UNNEST(user_properties) AS user_properties
    WHERE 
        ((SELECT event_params.value.string_value
        FROM UNNEST(event_params) AS event_params
        WHERE event_params.key = 'firebase_screen_class') = 'HomeActivity')
        GROUP BY
            user_properties.value.string_value) AS t2
ON t2.username = user_properties.value.string_value
WHERE
    t1.event_timestamp < t2.done_timestamp
    AND t1.event_name = 'screen_view'
    AND t2.username != 'null'
    AND ((SELECT event_params.value.string_value
        FROM UNNEST(t1.event_params) AS event_params
        WHERE event_params.key = 'firebase_screen_class') != 
            (SELECT event_params.value.string_value
            FROM UNNEST(t1.event_params) AS event_params
            WHERE event_params.key = 'firebase_previous_class'))
"""
df = client.query(query).to_dataframe()
df.head(10)

,Date,event_name,user_pseudo_id,Username,timestamp,previous_timestamp,first_touch_timestamp,done_timestamp,Screen
0,20180918,screen_view,e055d13a04995d499cb582db0cdd9861,vinsep18,1537277652904038,1537277643534038,1537277456010000,1537277721949068,QuestionsActivity
1,20180918,screen_view,e055d13a04995d499cb582db0cdd9861,vinsep18,1537277667756043,1537277653957043,1537277456010000,1537277721949068,QuestionsActivity
2,20180918,screen_view,e055d13a04995d499cb582db0cdd9861,vinsep18,1537277680732049,1537277668983049,1537277456010000,1537277721949068,QuestionsActivity
3,20180918,screen_view,e055d13a04995d499cb582db0cdd9861,vinsep18,1537277695147055,1537277681927055,1537277456010000,1537277721949068,QuestionsActivity
4,20180918,screen_view,e055d13a04995d499cb582db0cdd9861,vinsep18,1537277710624061,1537277697357061,1537277456010000,1537277721949068,QuestionsActivity
5,20180918,screen_view,e055d13a04995d499cb582db0cdd9861,vinsep18,1537277721523067,1537277711793067,1537277456010000,1537277721949068,QuestionsActivity
6,20180918,screen_view,e055d13a04995d499cb582db0cdd9861,vinsep18,1537277492534010,1537277488612010,1537277456010000,1537277721949068,DexterActivity
7,20180918,screen_view,f29429561128e4ac35ace6435278b8f5,emily,1537276898440011,1537276890991011,1537276848519000,1537276933746019,DexterActivity
8,20180918,screen_view,e055d13a04995d499cb582db0cdd9861,vinsep18,1537277483984001,1537277459757001,1537277456010000,1537277721949068,SignupActivity
9,20180918,screen_view,e055d13a04995d499cb582db0cdd9861,vinsep18,1537277486506004,1537277484011004,1537277456010000,1537277721949068,WelcomeActivity


### Post-Onboarding Screen View Timing

In [24]:
query = """
SELECT
    t1.event_date AS Date,
    t1.event_name AS event_name,
    t2.username as Username,
    t1.event_timestamp AS timestamp,
    t1.event_previous_timestamp AS previous_timestamp,
    t1.user_first_touch_timestamp AS first_touch_timestamp,
    (SELECT event_params.value.string_value
        FROM UNNEST(t1.event_params) AS event_params
        WHERE event_params.key = 'firebase_previous_class') AS Screen
FROM
    `analytics_153084895.events_*` as t1,
    UNNEST(t1.user_properties) AS user_properties
JOIN(
    SELECT
        MIN(event_timestamp) AS done_timestamp,
        user_properties.value.string_value AS username
    FROM
        `analytics_153084895.events_*`,
        UNNEST(user_properties) AS user_properties
    WHERE 
        ((SELECT event_params.value.string_value
        FROM UNNEST(event_params) AS event_params
        WHERE event_params.key = 'firebase_screen_class') = 'HomeActivity')
    GROUP BY
        user_properties.value.string_value) AS t2
ON t2.username = user_properties.value.string_value
WHERE
    t1.event_timestamp > t2.done_timestamp
    AND t1.event_name = 'screen_view'
    AND t2.username != 'null'
    AND ((SELECT event_params.value.string_value
        FROM UNNEST(t1.event_params) AS event_params
        WHERE event_params.key = 'firebase_screen_class') != 
            (SELECT event_params.value.string_value
            FROM UNNEST(t1.event_params) AS event_params
            WHERE event_params.key = 'firebase_previous_class'))
"""
df = client.query(query).to_dataframe()
df.head(10)

,Date,event_name,Username,timestamp,previous_timestamp,first_touch_timestamp,Screen
0,20181014,screen_view,test23,1539556129146026,1539556126726026,1537833795897000,SigninSignupActivity
1,20181224,screen_view,mckay68,1545628881089087,1545628876125087,1545628050055000,HomeActivity
2,20181224,screen_view,mckay68,1545628934517090,1545628881093090,1545628050055000,AboutYouActivity
3,20181224,screen_view,mckay68,1545628936442093,1545628934562093,1545628050055000,HomeActivity
4,20181224,screen_view,mckay68,1545628959003096,1545628936444096,1545628050055000,StrategiesActivity
5,20181224,screen_view,mckay68,1545628966846099,1545628959042099,1545628050055000,HomeActivity
6,20181224,screen_view,mckay68,1545628983581103,1545628966918103,1545628050055000,RelationshipsActivity
7,20181224,screen_view,mckay68,1545628996006108,1545628995842108,1545628050055000,RelationshipsActivity
8,20181224,screen_view,mckay68,1545628998367111,1545628996008111,1545628050055000,HomeActivity
9,20181224,screen_view,mckay68,1545629010970115,1545628998436115,1545628050055000,RelationshipsActivity


### Screen View Timing Data

In [25]:
query = """
SELECT
    (SELECT event_params.value.string_value
        FROM UNNEST(event_params) AS event_params
        WHERE event_params.key = 'firebase_previous_class') AS Screen,
    event_date AS Date,
    user_properties.value.string_value AS Username,
    event_timestamp AS end_timestamp,
    event_previous_timestamp AS start_timestamp
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE 
    event_name = 'screen_view'
    AND ((SELECT event_params.value.string_value
        FROM UNNEST(event_params) AS event_params
        WHERE event_params.key = 'firebase_screen_class') != 
            (SELECT event_params.value.string_value
            FROM UNNEST(event_params) AS event_params
            WHERE event_params.key = 'firebase_previous_class'))
ORDER BY
    event_timestamp
"""
df = client.query(query).to_dataframe()
df.head(10)

,Screen,Date,Username,end_timestamp,start_timestamp
0,HomeActivity,20180711,None,1531358521175000,1531257346796000
1,SigninSignupActivity,20180711,None,1531358531292002,1531358521175002
2,SignupActivity,20180711,None,1531358536305003,1531358531292003
3,SigninSignupActivity,20180711,None,1531358536478004,1531358536305004
4,SignupActivity,20180711,None,1531358937284006,1531358536478006
5,HomeActivity,20180712,None,1531376260465000,1531259406844000
6,SigninSignupActivity,20180712,None,1531376262537002,1531376260465002
7,SigninActivity,20180712,None,1531376312750005,1531376262537005
8,HomeActivity,20180712,None,1531376320627007,1531376312750007
9,SettingsActivity,20180712,None,1531376325091009,1531376320627009


### SMS vs MMS

In [26]:
query = """
SELECT
    type as type,
    COUNT(username) AS events
FROM
    (SELECT
        (SELECT event_params.value.string_value
            FROM UNNEST(event_params) AS event_params
            WHERE event_params.key = 'type') AS type,
        event_date AS date,
        user_properties.value.string_value AS username,
        event_timestamp AS timestamp,
        event_previous_timestamp AS previous_timestamp
    FROM
        `analytics_153084895.events_*`,
         UNNEST(user_properties) AS user_properties
    WHERE 
        event_name = 'text_message_received' OR event_name = "risky_sent_sms_warning_event")
GROUP BY
    type
"""
df = client.query(query).to_dataframe()
df.head(10)

,type,events
0,sms,63343
1,mms,9665


In [ ]:
query = """
Template
"""
df = client.query(query).to_dataframe()
df.head(10)

In [ ]:
query = """
Template
"""
df = client.query(query).to_dataframe()
df.head(10)